In [195]:
from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from torch.utils.data import DataLoader, TensorDataset, Subset, Dataset
from sklearn.model_selection import train_test_split, KFold
from PIL import Image
import shutil

cudnn.benchmark = True
plt.ion()   # interactive mode
#https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

In [196]:
test_size = 0.25

dataset = np.load('data_train.npy').reshape((300, 300, 9032))
labels = np.load('t_train_corrected.npy')
if os.path.exists("train"):
    shutil.rmtree("train")
if os.path.exists("test"):
    shutil.rmtree("test")
os.mkdir("train")
os.mkdir("test")
os.mkdir("train/images")
os.mkdir("train/labels")
os.mkdir("test/images")
os.mkdir("test/labels")

train_indices, test_indices = train_test_split(range(9032), test_size=test_size, shuffle=False)

for i in range(10):
    for idx in np.where(labels[train_indices] == i)[0]:
        image = Image.fromarray(dataset[:, :, idx], mode = 'L').convert("RGB").resize((320, 320))
        image.save("train/images/" + str(i) + "_" + str(idx) + ".jpg")
        with open("train/labels/" + str(i) + "_" + str(idx) + ".txt", "w") as f:
            f.write(str(i) + " 0.5 0.5 1 1")
    for idx in np.where(labels[test_indices] == i)[0]:
        image = Image.fromarray(dataset[:, :, idx + len(train_indices)], mode = 'L').convert("RGB").resize((320, 320))
        image.save("test/images/" + str(i) + "_" + str(idx + len(train_indices)) + ".jpg")
        with open("test/labels/" + str(i) + "_" + str(idx + len(train_indices)) + ".txt", "w") as f:
            f.write(str(i) + " 0.5 0.5 1 1")
            
for idx in np.where(labels[train_indices] == -1)[0]:
    image = Image.fromarray(dataset[:, :, idx], mode = 'L').convert("RGB").resize((320, 320))
    image.save("train/images/" + str(10) + "_" + str(idx) + ".jpg")
    with open("train/labels/" + str(10) + "_" + str(idx) + ".txt", "w") as f:
        f.write(str(10) + " 0.5 0.5 1 1")
for idx in np.where(labels[test_indices] == -1)[0]:
    image = Image.fromarray(dataset[:, :, idx + len(train_indices)], mode = 'L').convert("RGB").resize((320, 320))
    image.save("test/images/" + str(10) + "_" + str(idx + len(train_indices)) + ".jpg")
    with open("test/labels/" + str(10) + "_" + str(idx + len(train_indices)) + ".txt", "w") as f:
        f.write(str(10) + " 0.5 0.5 1 1")

In [197]:
def test_model(model, criterion, optimizer, scheduler):
    since = time.time()

    # Each epoch has a training and validation phase
    model.eval()

    running_loss = 0.0
    running_corrects = 0

    # Iterate over data.
    for inputs, labels in dataloader_test:
        labels = labels.type(torch.LongTensor) 

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward
        outputs = model(inputs)
        outputs.print()
        #_, preds = torch.max(outputs, 1)
        #print(preds)
        loss = criterion(outputs, labels)

        # statistics
        running_loss += loss.item() * inputs.size(0)
        running_corrects += torch.sum(preds == labels.data)

    loss = running_loss / dataset_test_size
    acc = running_corrects.double() / dataset_test_size

    print(f'Loss: {loss:.4f} Acc: {acc:.4f}')

    time_elapsed = time.time() - since
    print(f'Testing complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')

    return model

In [198]:
import os
import pandas as pd
from torchvision.io import read_image

class CustomSymbolImageDataset(Dataset):
    def __init__(self, img_dir, img_labels, img_root):
        self.img_dir = img_dir
        self.img_labels = img_labels
        self.img_root = img_root

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_root, self.img_dir[idx])
        image = read_image(img_path)
        label = self.img_labels[idx]
        return image, label

In [199]:
test_root = "test/images"
test_labels = np.array([int(img[0]) if img[1] != '_' else 10 for img in os.listdir(test_root)])
test_imgs = os.listdir(test_root)
test_dataset = CustomSymbolImageDataset(test_imgs, test_labels, test_root)
#device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model_ft = torch.hub.load('ultralytics/yolov5', 'custom', path='best.pt', force_reload=True)
#model_ft = model_ft.to(device)

Downloading: "https://github.com/ultralytics/yolov5/archive/master.zip" to C:\Users\justin.rossiter/.cache\torch\hub\master.zip
requirements: YOLOv5 requirements "gitpython" "tqdm>=4.64.0" not found, attempting AutoUpdate...
requirements:  Command 'pip install "gitpython" "tqdm>=4.64.0"  ' returned non-zero exit status 1.
YOLOv5  2022-11-27 Python-3.8.8 torch-1.9.0+cu111 CUDA:0 (NVIDIA GeForce RTX 3090, 24576MiB)

Fusing layers... 
Model summary: 157 layers, 7039792 parameters, 0 gradients
Adding AutoShape... 


In [194]:
dataloader_test = DataLoader(test_dataset, batch_size = 4, shuffle = True, num_workers = 0)
dataset_test_size = len(test_dataset)
criterion = nn.CrossEntropyLoss()

# Observe that all parameters are being optimized
optimizer_ft = optim.SGD(model_ft.parameters(), lr=0.001, momentum=0.9)

# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=7, gamma=0.1)
test_model(model_ft, criterion, optimizer_ft, exp_lr_scheduler)

AttributeError: 'Tensor' object has no attribute 'print'